In [28]:
%load_ext autoreload
from google.auth import default
credentials, project = default()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:

%autoreload 2
import os
import sys
from pathlib import Path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from dotenv import load_dotenv
load_dotenv()

from src.config_loader import config
from src.gcs_storage import read_from_gcs, upload_to_gcs
from src.utils import get_first_n_items
from src.wiktionary import generate_wiktionary_links
from tqdm import tqdm
config.TARGET_LANGUAGE_NAME

'French'

In [26]:
generate_wiktionary_links(phrase="Tout d'un coup, tout est devenu limpide.")

'<a href="https://en.wiktionary.org/wiki/tout#French">Tout</a> d\'un <a href="https://en.wiktionary.org/wiki/coup#French">coup,</a> <a href="https://en.wiktionary.org/wiki/tout#French">tout</a> <a href="https://en.wiktionary.org/wiki/est#French">est</a> <a href="https://en.wiktionary.org/wiki/devenu#French">devenu</a> <a href="https://en.wiktionary.org/wiki/limpide#French">limpide.</a>'

# Wiktionary Links

```raw
These are stored as a space separated string of either the words or a <a href> <\a> item.

Within the phrase dictionary they are stored with the key 'wiktionary_link'

phrase_key : {'english' : phrase key,
            'french' : phrase key,
            'wiktionary_links' : <a href>phrase</a> key}```

We store the wiktionary links as a cache so they can be reused in, e.g. the stories

In [35]:
# get current translated JSON file
from src.gcs_storage import get_wiktionary_cache_path

TRANSLATION_FILE = F"collections/LM1000/translations/{config.TARGET_LANGUAGE_NAME.lower()}.json"
WORD_LINK_CACHE_FILE = get_wiktionary_cache_path() 
phrase_translations = read_from_gcs(bucket_name=config.GCS_PRIVATE_BUCKET, file_path=TRANSLATION_FILE)
word_link_cache = read_from_gcs(bucket_name=config.GCS_PRIVATE_BUCKET, file_path=WORD_LINK_CACHE_FILE)

In [36]:
phrase_translations

{'he_might_offer_to_help_with_the_work': {'english': 'He might offer to help with the work',
  'french': 'Il pourrait proposer de donner un coup de main',
  'wiktionary_links': '<a href="https://en.wiktionary.org/wiki/il#French">Il</a> <a href="https://en.wiktionary.org/wiki/pourrait#French">pourrait</a> <a href="https://en.wiktionary.org/wiki/proposer#French">proposer</a> <a href="https://en.wiktionary.org/wiki/de#French">de</a> <a href="https://en.wiktionary.org/wiki/donner#French">donner</a> <a href="https://en.wiktionary.org/wiki/un#French">un</a> <a href="https://en.wiktionary.org/wiki/coup#French">coup</a> <a href="https://en.wiktionary.org/wiki/de#French">de</a> <a href="https://en.wiktionary.org/wiki/main#French">main</a>'},
 'i_know_exactly_what_youre_talking_about': {'english': "I know exactly what you're talking about",
  'french': 'Je sais exactement de quoi tu parles',
  'wiktionary_links': '<a href="https://en.wiktionary.org/wiki/je#French">Je</a> <a href="https://en.wikt

In [ ]:
test_dict = get_first_n_items(phrase_translations, 3)

In [31]:
from src.wiktionary import add_wiktionary_links
word_link_cache = add_wiktionary_links(phrase_translations, word_link_cache = word_link_cache, overwrite=True)

Adding wiktionary links: 100%|██████████| 830/830 [02:43<00:00,  5.07it/s]


In [32]:
len(word_link_cache)

2910

In [33]:
upload_to_gcs(obj=phrase_translations, bucket_name=config.GCS_PRIVATE_BUCKET, file_name=TRANSLATION_FILE)

'gs://audio-language-trainer-private-content/collections/LM1000/translations/french.json'

In [34]:
upload_to_gcs(word_link_cache, bucket_name=config.GCS_PRIVATE_BUCKET, file_name=WORD_LINK_CACHE_FILE)

'gs://audio-language-trainer-private-content/resources/french/french_wiktionary_cache.json'